# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [34]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [35]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [36]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [37]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [38]:
document = ET.parse( './data/mondial_database.xml' )

#### 1. 10 Countries with the Lowest Infant Mortality Rates

In [39]:
countries=[]

# Find country names and infant_mortality in the file
for element in document.iterfind('country'):
    country_list=element.find('name')
    infant_mortality=element.find('infant_mortality') 
    
# Extract country name and infant mortality where available
    if country_list != None:
        if infant_mortality != None:    

# Country name and infant_mortality data added to Countries list
            countries.append([country_list.text, infant_mortality.text])
            
# Countries list converted to a dataframe  
df=pd.DataFrame(countries, columns=['Country', 'Infant Mortality'])

# Infant Mortality column converted to numeric
df['Infant Mortality']=pd.to_numeric(df['Infant Mortality'])

# Ranking by lowest infant mortality
lowest10=df.set_index('Country').sort_values(by='Infant Mortality').head(10)
lowest10

,Infant Mortality
Country,
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13


#### 2. 10 Cities with the Largest Population

In [40]:
cities=[]

# Find city and province names in the file
for element in document.iterfind('country'):
    city_list=element.findall('city')
    prov_list=element.findall('province') 
    
# Add all cities in provinces to the city_list
    for prov in prov_list:
        city_list +=prov.findall('city')
    
# Find current population by city, extract, & add to list
    for city in city_list:
        city_name=city.find('name').text
        pop=city.find('.//population[last()]')

        if pop !=None:
            city_pop=int(pop.text)
         
        cities.append([city_name, city_pop])

# List converted to dataframe & sorted by highest population
citiesdf = pd.DataFrame(Cities, columns=['City', 'Population'])
citiesdf.sort_values(by='Population', ascending=False).head(10)


,City,Population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


#### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [41]:
groups=[]
countries2=[]

# Find country name, population, and ethnic group percentages, and add to groups list
for elem in document.iterfind('country'):
    country = elem.find('name').text
    pop = elem.find('population')

    for e in elem.iterfind('ethnicgroup'):
        grp = e.text
        per = float(e.attrib['percentage'])

        groups.append([country,grp,per]) 
    
# Add country and population to countries2 list
    for y in elem.iterfind('population'):
        pop = int(y.text)
        year = int(y.attrib['year'])
        countries2.append([country, year, pop])

# Convert groups list and countries2 list to dataframes
groupsdf = pd.DataFrame(groups, columns = ['Country', 'Ethnic Group', 'Percentage'])
countries2df = pd.DataFrame(countries2, columns = ['Country', 'Year', 'Population'])  

latest = countries2df.groupby(['Country'])['Year'].transform(max) == countries2df['Year']
countries2df = countries2df[latest]

# Merge dataframes on Countries column
groupspop = countries2df.merge(groupsdf, how='inner')

# Calculate the actual population for each Ethnic Group by Country
groupspop['Ethnic Group Population'] = (groupspop.Percentage/100)*groupspop.Population

# Add the Ethnic Group Populations and sort highest to lowest
groupspop.groupby(['Ethnic Group'])['Ethnic Group Population'].sum().sort_values(ascending=False).head(10)

Ethnic Group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Ethnic Group Population, dtype: float64

#### 4. Name and country of a) longest river, b) largest lake and c) airport at highest elevation